## Data/Text Mining Project


Course: CISD 43<br>
Professor: Sohair Zaki<br>
Student: Fiona Xu and Jack Chen<br>
<br>
Purpose:<br>
- hello world
<br>

Details:<br>
1. Load libraries, data cleaning<br>
2. Exploratory Data Analysis (EDA)<br>
3. Deep Learning<br>
4. Conclusion<br>
5. References<br>

## Libraries, Functions, Load/Cleaning Data

### Load Libraries

In [ ]:
# Parsing Part
import requests
from bs4 import BeautifulSoup
# Alpha Vantage API - stock data
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from alpha_vantage.techindicators import TechIndicators

# Data
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
sns.set_style("whitegrid")

# Deep Learning
import tensorflow
import keras.layers

# Data Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Model Featuring
from keras.layers import Input, Flatten, Dense, Activation, LeakyReLU, Bidirectional, LSTM
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor

# Models
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression
from keras import Sequential, Model

# Analyze Results
from sklearn.metrics import roc_curve, plot_roc_curve, mean_absolute_error, mean_squared_error, accuracy_score

# Misc
# filter warnings
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History   
import warnings
warnings.filterwarnings("ignore")
import pickle

### Functions

In [ ]:
# Function to clean balance sheet df
def convMonth(x):
    if x == 'MAR':
        x = 3
    elif x == 'DEC':
        x = 12
    elif x =='JUN':
        x = 6
    elif x == 'SEP':
        x = 9
    return x

In [ ]:
# Function - ML - run machine learning model and print out MSE
def runML(model,feature,target,ftest,ttest):
    model.fit(feature, target)
    print(model)
    print('MAE for training:', mean_absolute_error(target,model.predict(feature)))
    print('MAE for testing:', mean_absolute_error(ttest,model.predict(ftest)))
    print('R2 score:', model.score(ftest,ttest))
    print('')

In [ ]:
# Convert to sequence function for TimeSeries to seq models 
def convSeq(feature, target, seq):
    Xs, ys = [],[]
    
    # from 0 to (len - seq)
    for i in range(len(feature) - seq):

        # from i to (i + seq)
        v = feature.iloc[i:(i+seq)]
        Xs.append(v)

        # target of (i+seq)
        ys.append(target.iloc[i+seq])
    return np.array(Xs), np.array(ys)

In [ ]:
# Convert sequence dimenstions
def convDim(feature, target):
    targetArray = np.array(target)
    targetNew = targetArray.reshape(-1,1)
    featureArray = np.array(feature)
    featureNew = featureArray.reshape(featureArray.shape[0], featureArray.shape[1], 1)
    return featureNew, targetNew

In [ ]:
# didnt work...

def hyperLSTM(units=32,activation='relu',optimizer='adam'):
    x=Xt
    ls = Sequential()
    ls.add(LSTM(units=32,activation=activation,return_sequences=True,input_shape=(x[1],x[2])))
    ls.add(LSTM(units=32/2, activation=activation, input_shape=(x[1],units)))
    ls.add(Dense(units=3, activation = activation))
    ls.add(Dense(units=1, activation = activation))
    ls.compile(optimizer=optimizer, loss='mse',metrics=['mse','mae'])
    return ls

### AlphaVantage API

In [ ]:
# Variable - store Alpha Vantage API key
# path =  r'C:\Users\Gumo\Desktop\Git\Notebook\keys\alphaVantage.txt'
# with open(path, mode='r') as w:
#     key = (w.readline())

In [ ]:
# URL Method
# def alpha(function, symbol):
#     url = 'https://www.alphavantage.co/query?function='+function+'&symbol='+symbol+'&apikey='+key
#     response = requests.get(url)

# Function AlphaVantage
def alpha(symbol, function, period=None, typ = None, tim=None):
    
    # Funciton - TimeSeries
    if function == 't':
        # Instance - TimeSeries Instance
        ts = TimeSeries(key = key, output_format='pandas')
        if period == 'd':
            data = ts.get_daily_adjusted(symbol, outputsize=outputsize)[0]
        elif period == 'w':
            data = ts.get_weekly_adjusted(symbol)[0]
        elif period == 'm':
            data = ts.get_monthly_adjusted(symbol)[0]

    # Funciton - FundamentalData
    elif function == 'f':
        # Instance - FundamentalData Instance
        fd = FundamentalData(key, output_format='pandas')
        if period == 'q':
            if typ == 'i':
                data = fd.get_income_statement_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ == 'b':
                data = fd.get_balance_sheet_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ=='c':
                data = fd.get_cash_flow_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
        elif period =='a':
            if typ == 'i':
                data = fd.get_income_statement_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ == 'b':
                data = fd.get_balance_sheet_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ=='c':
                data = fd.get_cash_flow_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
    
    # Funciton - TechnicalIndicator            
    elif function =='ti':
        ti = TechIndicators(key,output_format='pandas')

        # reassign period into TI format
        if period == 'm':
            period = 'monthly'
        elif period =='w':
            period = 'weekly'
        elif period == 'd':
            period ='daily'

        if typ == 'rsi':
            data=ti.get_rsi(symbol,interval=period,time_period=tim,series_type='close')[0]
        elif typ =='so':
            data = ti.get_stoch(symbol,interval=period)[0]
            
    # Funciton - CC
    elif function == 'c':
        # Instance - Crypto
        cc = CryptoCurrencies(key, output_format='pandas')
        pass
    return data

In [ ]:
# Variable Size
outputsize = 'compact'

# Variable Stock
symbol = 'cvx'

# Variable Function - t, f, c, ti (timeseries, financialdata, crypto, technical indicator)
function = 'ti'

# Variable Period
# TimeSeries: d,w,m,i (daily, weekly, monthly, interval(mins))
# Fundamental: q,a (quarterly,annually)
# Technical: d,w,m, (interval(mins))
period = 'm'

# Varible - Type
# Fundamental: i,b,c
# Technical: so,rsi
typ ='so'

# Varible - Time
tim = 60

info = alpha(symbol,function,period=period,typ = typ,tim=tim)
info.head()

In [ ]:
# save to local stock data
# info.to_csv('data/'+symbol.upper()+'stockPrice.csv')

### BeautifulSoup

In [ ]:
# Define a variable for the url of the site
site = "https://www.usinflationcalculator.com/inflation/current-inflation-rates/#:~:text=The%20annual%20inflation%20rate%20for,at%208%3A30%20a.m.%20ET"

In [ ]:
# Making a get request and assign the result to a variable response
response = requests.get(site)

#Check that the response was processed correctly
response.status_code

200

In [ ]:
# Extracting the HTML
#assign a variable html to response content.
html = response.content

# Checking that the reply is indeed an HTML code by inspecting the first 200 symbols
html[:200]

b'<!DOCTYPE html>\n<!--[if IE 7]>\n<html class="ie ie7" dir="ltr" lang="en-US"\n\tprefix="og: https://ogp.me/ns#" >\n<![endif]-->\n<!--[if IE 8]>\n<html class="ie ie8" dir="ltr" lang="en-US"\n\tprefix="og: https'

In [ ]:
#Convert HTML to a BeautifulSoup object, using the default parser of html  
#Create a BeautifulSoup object and store it in a variable named soup.
soup = BeautifulSoup(html, "html.parser")

In [ ]:
# The soup variable (BeautifulSoup object) we defined earlier can be seen as representing the whole document
soup

<!DOCTYPE html>

<!--[if IE 7]>
<html class="ie ie7" dir="ltr" lang="en-US"
	prefix="og: https://ogp.me/ns#" >
<![endif]-->
<!--[if IE 8]>
<html class="ie ie8" dir="ltr" lang="en-US"
	prefix="og: https://ogp.me/ns#" >
<![endif]-->
<!--[if !(IE 7) | !(IE 8) ]><!-->
<html dir="ltr" lang="en-US" prefix="og: https://ogp.me/ns#">
<!--<![endif]-->
<head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-2181571-7"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-2181571-7');
</script>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://www.usinflationcalculator.com/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
	<script src="https://www.usinflationcalculator.com/wp-content/themes/twentyfourteen/js/html5.js"></script>


In [ ]:
#using prettify() method to turn a Beautiful Soup parse tree into a nicely formatted Unicode string,
#with a separate line for each tag and each string:
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 7]>
<html class="ie ie7" dir="ltr" lang="en-US"
	prefix="og: https://ogp.me/ns#" >
<![endif]-->
<!--[if IE 8]>
<html class="ie ie8" dir="ltr" lang="en-US"
	prefix="og: https://ogp.me/ns#" >
<![endif]-->
<!--[if !(IE 7) | !(IE 8) ]><!-->
<html dir="ltr" lang="en-US" prefix="og: https://ogp.me/ns#">
 <!--<![endif]-->
 <head>
  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-2181571-7">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-2181571-7');
  </script>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://www.usinflationcalculator.com/xmlrpc.php" rel="pingback"/>
  <!--[if lt IE 9]>
	<script src="https://www.usinflationcalculator.com/wp-content/themes/twentyfourteen/

In [ ]:
# Exporting the HTML to a file
with open('output/inflation_rate_response.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

In [ ]:
# creating list with all tables
tables = soup.find_all('table')

In [ ]:
# Inspect the value of the variable
tables

[<table cellpadding="0" cellspacing="0" width="110%">
 <tr height="17">
 <td align="right" height="17"><strong>Year</strong></td>
 <td align="right"><strong>Jan</strong></td>
 <td align="right"><strong>Feb</strong></td>
 <td align="right"><strong>Mar</strong></td>
 <td align="right"><strong>Apr</strong></td>
 <td align="right"><strong>May</strong></td>
 <td align="right"><strong>Jun</strong></td>
 <td align="right"><strong>Jul</strong></td>
 <td align="right"><strong>Aug</strong></td>
 <td align="right"><strong>Sep</strong></td>
 <td align="right"><strong>Oct</strong></td>
 <td align="right"><strong>Nov</strong></td>
 <td align="right"><strong>Dec</strong></td>
 <td align="right"><strong>Ave</strong></td>
 </tr>
 <tr>
 <td align="right" height="17"><strong>2022</strong></td>
 <td align="right">7.5</td>
 <td align="right">7.9</td>
 <td align="right">8.5</td>
 <td align="right">8.3</td>
 <td align="right"><em>Avail.<br/>
      June<br/>
      10</em></td>
 <td align="right"> </td>
 <td a

In [ ]:
#since there is only one table, set table as the first table in tables
table = tables[0]
#display table
table

<table cellpadding="0" cellspacing="0" width="110%">
<tr height="17">
<td align="right" height="17"><strong>Year</strong></td>
<td align="right"><strong>Jan</strong></td>
<td align="right"><strong>Feb</strong></td>
<td align="right"><strong>Mar</strong></td>
<td align="right"><strong>Apr</strong></td>
<td align="right"><strong>May</strong></td>
<td align="right"><strong>Jun</strong></td>
<td align="right"><strong>Jul</strong></td>
<td align="right"><strong>Aug</strong></td>
<td align="right"><strong>Sep</strong></td>
<td align="right"><strong>Oct</strong></td>
<td align="right"><strong>Nov</strong></td>
<td align="right"><strong>Dec</strong></td>
<td align="right"><strong>Ave</strong></td>
</tr>
<tr>
<td align="right" height="17"><strong>2022</strong></td>
<td align="right">7.5</td>
<td align="right">7.9</td>
<td align="right">8.5</td>
<td align="right">8.3</td>
<td align="right"><em>Avail.<br/>
     June<br/>
     10</em></td>
<td align="right"> </td>
<td align="right"> </td>
<td alig

In [ ]:
# extracting all rows
table.find_all('tr')

[<tr height="17">
 <td align="right" height="17"><strong>Year</strong></td>
 <td align="right"><strong>Jan</strong></td>
 <td align="right"><strong>Feb</strong></td>
 <td align="right"><strong>Mar</strong></td>
 <td align="right"><strong>Apr</strong></td>
 <td align="right"><strong>May</strong></td>
 <td align="right"><strong>Jun</strong></td>
 <td align="right"><strong>Jul</strong></td>
 <td align="right"><strong>Aug</strong></td>
 <td align="right"><strong>Sep</strong></td>
 <td align="right"><strong>Oct</strong></td>
 <td align="right"><strong>Nov</strong></td>
 <td align="right"><strong>Dec</strong></td>
 <td align="right"><strong>Ave</strong></td>
 </tr>, <tr>
 <td align="right" height="17"><strong>2022</strong></td>
 <td align="right">7.5</td>
 <td align="right">7.9</td>
 <td align="right">8.5</td>
 <td align="right">8.3</td>
 <td align="right"><em>Avail.<br/>
      June<br/>
      10</em></td>
 <td align="right"> </td>
 <td align="right"> </td>
 <td align="right"> </td>
 <td ali

In [ ]:
#Gets all the column headers of our table
#create an empty list name it headers
headers = []
#loop through the first row which containts the header names
#save the text in the table header tag after stripping any extra spaces as title
#append this title to the headers list
for i in table.find_all('tr')[0]:
    title = i.text.strip()
    if title != '':
        headers.append(title)
        
#display headers in a list
headers

In [ ]:
#Creates a dataframe using the column headers from our table
df = pd.DataFrame(columns = headers)

#display the header in a dataframe
df

In [ ]:
#create a for loop to iterate all the rows in the table
#retrieve the text in the row which is the value we want
#get the length of the dataframe
#populate the contents of the dataframe iteratively by reading each row 
#increment the length of the dataframe and add the row to the end of the dataframe
for j in table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [tr.text for tr in row_data]
    length = len(df)
    df.loc[length] = row

In [ ]:
#display the dataframe
df.head()

In [ ]:
# set year, month and inflaiton rate as the column name and remove unneeded value
months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
df_inflation = pd.melt(df, id_vars = 'Year', value_vars=months,var_name='Month', value_name='Inflation Rate')
df_inflation['Month']=pd.Categorical(df_inflation['Month'], categories=months, ordered=True)
df_inflation.sort_values(by=['Year','Month'], inplace=True)

In [ ]:
# display the head of the dataframe
df_inflation.head()

In [ ]:
# save it to a csv file
# df_inflation.to_csv('data/inflation_rate_clean.csv')

Scrape the Gas Price Website

In [ ]:
# Define a variable for the url of the site
site_gas = "https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=pet&s=emm_epm0_pte_nus_dpg&f=m"

In [ ]:
# Making a get request and assign the result to a variable response
response_gas = requests.get(site_gas)

#Check that the response was processed correctly
response_gas.status_code

In [ ]:
# Extracting the HTML
#assign a variable html to response content.
html_gas = response_gas.content

# Checking that the reply is indeed an HTML code by inspecting the first 200 symbols
html_gas[:500]

In [ ]:
#Convert HTML to a BeautifulSoup object, using the default parser of html  
#Create a BeautifulSoup object and store it in a variable named soup.
soup_gas = BeautifulSoup(html_gas, "html.parser")

In [ ]:
#using prettify() method to turn a Beautiful Soup parse tree into a nicely formatted Unicode string,
#with a separate line for each tag and each string:
print(soup_gas.prettify())

In [ ]:
# Exporting the HTML to a file
with open('output/gas_price.html', 'wb') as file:
    file.write(soup_gas.prettify('utf-8'))

In [ ]:
# creating list with all tables
tables_gas = soup_gas.find_all('table')

In [ ]:
tables_gas

In [ ]:
print('Classes of each table:')
for table in soup_gas.find_all('table'):
    print(table.get('class'))

In [ ]:
table_gas = soup_gas.find('table', class_='FloatTitle')

In [ ]:
table_gas

In [ ]:
# extracting all rows
table_gas.find_all('tr')

In [ ]:
# Inspecting the contents of first row
table_gas.find_all('tr')[0].contents

In [ ]:
#Gets all the column headers of our table
#create an empty list name it headers
headers_gas = []
#loop through the first row which containts the header names
#save the text in the table header tag after stripping any extra spaces as title
#append this title to the headers list
for i in table_gas.find_all('th'):
    title = i.text.strip()
    if title != '':
        headers_gas.append(title)
        
#display headers in a list
headers_gas

In [ ]:
#Creates a dataframe using the column headers from our table
df_gas = pd.DataFrame(columns = headers_gas[:14])

#display the header in a dataframe
df_gas

In [ ]:
# Get all the rows of table_gas
# create an empty table that will store all the data 
table_data=[]
for tr in table_gas.find_all('tr'):
    t_row={}
    #Each table row is stored in the form of 
    #t_row= {'Year': '', 'Jan': '', 'Feb': ''......'Dec': ''}
    
    #find all td's(13) in tr and zip it with headers_gas
    for td, th in zip(tr.find_all('td'), headers_gas):
        t_row[th] = td.text.strip()
    table_data.append(t_row)

In [ ]:
# display table_data
table_data

In [ ]:
# create a dataframe called df_gas and store table_data in it
df_gas = pd.DataFrame(table_data)

# display df_gas
df_gas

In [ ]:
#remove all the rows that contain null values
df_gas = df_gas.dropna()
df_gas

In [ ]:
# drop data in year 1993-1999 and Apr in 2022
df_gas = df_gas.drop([1, 2, 4, 5, 6, 7, 8])

In [ ]:
# change the format of the dataframe
months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

df_gas_new = pd.melt(df_gas, id_vars = 'Year', value_vars=months, var_name='Month', value_name='Gas Price')

df_gas['Month']=pd.Categorical(df_gas['Month'], categories=months, ordered=True)
df_gas.sort_values(by=['Year','Month'], inplace=True)
df_gas

### Load/Clean Data

In [ ]:
# load dfs - data will be 01/2000 to 03/2022, 22 years and 3 month is 267 month, 
df_bs = pd.read_excel('data/df_cvx_bs_quarter.xlsx', engine='openpyxl')
df_is = pd.read_excel('data/df_cvx_is_quarter.xlsx', engine='openpyxl')
df_stock = pd.read_csv('data/CVXstockPrice.csv')
df_inflation = pd.read_csv('data/clean_inflation_rate.csv')
df_gas = pd.read_excel('data/gasPrices.xlsx', engine='openpyxl')

# colab load
df_bs = pd.read_excel('df_cvx_bs_quarter.xlsx', engine='openpyxl')
df_is = pd.read_excel('df_cvx_is_quarter.xlsx', engine='openpyxl')
df_stock = pd.read_csv('CVXstockPrice.csv')

# read csv file into a dataframe
df_merge = pd.read_csv('data/final_df.csv', index_col=0)

In [ ]:
###

### Clean merged dataframe

In [ ]:
# change the column names
df_merge.rename(columns = {'adjustedClose':'AdjustedClose', 'volume':'Volume', 'year':'Year', 'month':'Month', 
                           'sales':'Sales', 'netIncome':'NetIncome','assets':'Assets', 'equity':'Equity','inflation':'InflationRate',
                           'gas':'GasPrice'}, inplace = True)


In [ ]:
# show the renamed dataframe
df_merge

In [ ]:
# reorder the column
df_merge=df_merge[['Year', 'Month', 'InflationRate', 'GasPrice', 'Sales', 'NetIncome', 'Assets', 'Equity',
             'Volume', 'AdjustedClose']]
# display the final dataframe
df_merge.head()

In [ ]:
# change the volume unit to million 
df_merge.Volume = df_merge.Volume/1000000
df_merge

In [ ]:
#formatting float column
#change the value of sales column to two decimal digits 
df_merge[['Sales', 'NetIncome']]=df_merge[['Sales', 'NetIncome']].round(decimals=2)
# display the cleaned dataframe
df_merge

In [ ]:
# save it to a csv file
df_merge.to_csv('data/df_final.csv')

Stcok DF

In [ ]:
d # we just need adjusted close , volumn
dfstock1 = df_stock.iloc[:,[0,5,6]]

# create year column and month column
dfstock1['year'] = df_stock.date.apply(lambda x : x.split('-')[0])
dfstock1['month'] = df_stock.date.apply(lambda x : x.split('-')[1])
dfstock1.tail()

# drop useless columns
dfstock2 = dfstock1.iloc[2:-1,1:]
# reset index
dfstock3 = dfstock2.reset_index(drop=True)

# convert month into int
dfstock3.month = dfstock3.month.apply(lambda x: int(x))

#assign to finaldf
dfstockf = dfstock3.sort_values(by=['year','month'])
dfstockf

Balance Sheet DF

In [ ]:
# df_BalanceSheet - Clean Balance Sheet
# transpose df
df_bs1 = df_bs.T

# assign column names
df_bs1.columns = df_bs1.iloc[0,:]

# reset index and drop first row
df_bs2 = df_bs1.reset_index().iloc[1:,:]
df_bs2.head(3)

Unnamed: 0,index,Cash & Short-Term Investments,Cash Only,Total Short Term Investments,Short-Term Receivables,"Accounts Receivables, Net","Accounts Receivables, Gross",Bad Debt/Doubtful Accounts,Other Receivables,Inventories,...,Other Appropriated Reserves,Treasury Stock,Repurchased Stock Value,Total Shareholders' Equity,Accumulated Minority Interest,Total Equity,Total Liabilities & Shareholders' Equity,Per Share,Book Value per Share,Tangible Book Value per Share
1,MAR '22,12.02,11.99,0.03,23.26,23.2,23.51,-0.3,0.05,6.53,...,-3.71,-39.16,1.25,146.22,0.88,147.1,249.05,NaN,74.42,72.19
2,DEC '21,6.01,5.97,0.04,18.42,18.17,18.47,-0.3,0.25,6.31,...,-3.96,-41.46,0.75,139.07,0.87,139.94,239.54,NaN,72.06,69.79
3,SEP '21,6.28,6.24,0.03,16.57,16.16,16.45,-0.29,0.4,6.13,...,-4.51,-41.42,0.63,135.86,0.86,136.72,239.95,NaN,70.48,68.2


In [ ]:
# see the columns
df_bs2.columns

Index(['index', 'Cash & Short-Term Investments', 'Cash Only',
       'Total Short Term Investments', 'Short-Term Receivables',
       'Accounts Receivables, Net', 'Accounts Receivables, Gross',
       'Bad Debt/Doubtful Accounts', 'Other Receivables', 'Inventories',
       'Finished Goods', 'Raw Materials', 'Other Current Assets',
       'Miscellaneous Current Assets', 'Total Current Assets',
       'Net Property, Plant & Equipment',
       'Property, Plant & Equipment - Gross',
       'Operating Lease Right-of-Use Assets', 'Accumulated Depreciation',
       'Total Long-Term Investments ', 'LT Investment - Affiliate Companies',
       'Other Long-Term Investments', 'Long-Term Note Receivable',
       'Intangible Assets', 'Goodwill', 'Deferred Tax Assets', 'Other Assets',
       'Tangible Other Assets', 'Total Assets',
       'Liabilities & Shareholders' Equity', 'ST Debt & Curr. Portion LT Debt',
       'Accounts Payable', 'Income Tax Payable', 'Other Current Liabilities',
       'Misc

In [ ]:
# select the needed columns
df_bs3 = df_bs2[['index','Total Assets', 'Total Equity']].iloc[:89,:]
df_bs3.shape

Unnamed: 0,index,Total Assets,Total Equity
1,MAR '22,249.05,147.1
2,DEC '21,239.54,139.94
3,SEP '21,239.95,136.72
4,JUN '21,242.81,133.91
5,MAR '21,241.65,132.93


In [ ]:
# since data is quarterly, we will append the data until we have 267 month
dfbs4 = df_bs3
dfbs5 = dfbs4.append(df_bs3)
dfbs6 = dfbs5.append(df_bs3)
dfbs6.shape

In [ ]:
# convert year and month
dfbs6['year'] = dfbs6['index'].apply(lambda x : int(x.split(' \'')[1]) + 2000)
dfbs6['month'] = dfbs6['index'].apply(lambda x : convMonth(x.split(' \'')[0]))
dfbs6.head()

In [ ]:
dfbsf = dfbs6.sort_values(by=['year','month'])
dfbsf.shape

Income Statement DF

In [ ]:
# Income Statement - Clean Balance Sheet
# transpose df
df_is1 = df_is.T

# assign column names
df_is1.columns = df_is1.iloc[0,:]

# reset index and drop first row
df_is2 = df_is1.reset_index().iloc[1:,:]
df_is2.head(3)

In [ ]:
# see the columns
df_is2.columns

In [ ]:
# select needed columns
df_is3 = df_is2[['index','Sales','Net Income']].iloc[:89,:]
df_is3.shape

In [ ]:
# append df to 267 month
dfis4 = df_is3
dfis5 = dfis4.append(df_is3)
dfis6 = dfis5.append(df_is3)


In [ ]:
# 22 year 3 month = 267 month
dfis6.shape

In [ ]:
# create year and month columns
dfis6['year'] = dfis6['index'].apply(lambda x : int(x.split(' \'')[1]) + 2000)
dfis6['month'] = dfis6['index'].apply(lambda x : convMonth(x.split(' \'')[0]))
dfis6.head()

In [ ]:
# sort by values
dfisf = dfis6.sort_values(by=['year','month'])
dfisf.shape

Inflation DF

In [ ]:
# format inflation df
dfi1 = df_inflation.iloc[:267,:]
dfi1

Gas DF

In [ ]:
df_gas.Date = df_gas.Date.astype(str)

In [ ]:
df_gas['year'] = df_gas.Date.apply(lambda x : x.split('-')[0])
df_gas['month'] = df_gas.Date.apply(lambda x : x.split('-')[1])
df_gas

In [ ]:
# get dfgas1
dfgas1 = df_gas.iloc[81:,:]

In [ ]:
# create merged df
dfmerge = dfstockf

#rewrite quartly month into 12 month
dfmerge['sales'] = list(dfisf.Sales)
dfmerge['netIncome'] = list(dfisf['Net Income'])
dfmerge['Total Assets'] = list(dfbsf['Total Assets'])
dfmerge['Total Equity'] = list(dfbsf['Total Equity'])
dfmerge['inflation'] = list(dfi1['Inflation Rate'])
dfmerge['gas'] = list(dfgas1['U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)'])

# create the dfmerge1 reset index, rename columns
dfmerge1 = dfmerge.reset_index(drop=True).rename(columns={'5. adjusted close':'adjustedClose','6. volume':'volume','Total Assets': 'assets','Total Equity': 'equity'})

''' Units
sales - millions
netIncome - millions
asset - billions
equity - billions

'''

# assign new df
dfmerge2 = dfmerge1

# convert into millions
dfmerge2.equity = dfmerge1.equity * 1000
dfmerge2.assets = dfmerge1.assets * 1000
# convert into 3 month average
dfmerge2.sales = dfmerge1.sales/3
dfmerge2.netIncome = dfmerge1.netIncome/3
dfmerge2.year = dfmerge2.year.astype(int)
dfmerge2.inflation = dfmerge2.inflation.astype(float)

In [ ]:
dfmerge2.info()

In [ ]:
# dfmerge2.to_csv('data/final_df.csv')

## Exploratory Data Analysis (EDA)

In [ ]:
# Get the basic information about the data
df_merge.info()

In [ ]:
# Get the statistics of the data
df_merge.describe()

In [ ]:
# check the distribution of our target column
sns.histplot(df_merge['AdjustedClose'])

#### Conclusion:
It looks like the chevron stock price is not normally distributed.
The average price is falling somewhat around 60 dollars, with more price falling on 20 dollars.

In [ ]:
# We want to know the yearly sales change 
# we need to group the rows by year
df_merge.groupby(['Year']).sum()

In [ ]:
# let's plot the yearly sales change with a scatterplot
plt.figure(figsize=(15, 10))
sns.barplot(data=df_merge, x='Year', y='Volume')

#### Time Series Data Visualization

In [ ]:
#add a new column named date to df_merge 
#assign day as 1 so that the datatype of date becomes datetime
#this column is for implementing time series plotting
df_merge['Date']=pd.to_datetime(df_merge[['Year', 'Month']].assign(Day=1))

In [ ]:
#display the new dataframe
df_merge

In [ ]:
#create a new dataframe to store columns for time series analysis
#setting the Date as index
df_timeseries=df_merge[['Date','GasPrice', 'Sales', 'AdjustedClose']].set_index('Date')

#display the dataframe
df_timeseries

In [ ]:
# normalize all the column data and plot them together to see the different trends 
df_scaled = df_timeseries

# construct a scaler
scaler= MinMaxScaler()
df_scaled.iloc[:,:] = scaler.fit_transform(df_timeseries)

In [ ]:
df_scaled

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(df_scaled['GasPrice'])
plt.plot(df_scaled['Sales'])
plt.plot(df_scaled['AdjustedClose'])
plt.grid(axis='y')
plt.title('Chevron Adjusted Close from Year 2000 to 2022')
plt.xlabel('Date')
plt.ylabel('Chevron Adjusted Close (million dollars)')
plt.legend(['Inflation Rate', 'Gas Price', 'Sales', 'Adjusted Close'], prop={'size':20})
plt.show()

### Analyze the Relationship between Inflation and Stock Price
- Examine whether inflation rate can predict stock price
- Convert stock price to a percent change since inflation rate is a percent change

In [ ]:
# create a list for monthly data
df_merge['AdjustedCloseChange']=df_merge['AdjustedClose']/df_merge['AdjustedClose'].shift(1)*100 - 100
df_change=df_merge[['Date', 'AdjustedCloseChange', 'InflationRate']].set_index('Date')

In [ ]:
df_change

In [ ]:
# drop the first row that contains null value
df_change.drop(index='2000-01-01')

### Visualize the infaltion rate and stock price change and their relationships

In [ ]:
# set the style of the plot to be white
sns.set_style('white')

# set the figure size of the plot to be 15 width and 8 height
fig, ax1 = plt.subplots(figsize=(15, 8))

# create a twin axes sharing the xaxis
# create a new axes with an invisible x-axis and an independent y-axis positioned opposite to the original one
ax2 = ax1.twinx()

# plot inflation rate on ax1
ax1.plot(df_change.index, df_change.InflationRate,
         color="green", label="InfaltionRate")

#set positive adjusted close change as df_change_plus
df_change_plus=df_change[df_change['AdjustedCloseChange']>=0]

#position positive adjusted close change above x-axis
#set the line of color to be red
ax2.bar(df_change_plus.index, df_change_plus['AdjustedCloseChange'], 
       color='red', alpha=0.5, width=20, label='Chevron Adjusted Close(positive)')

#create a dataframe to store negative adjusted close change
df_change_minus = df_change[df_change['AdjustedCloseChange']<0]

#position negative adjusted close change below x-axis
#set the line of color to be blue
ax2.bar(df_change_minus.index, df_change_minus['AdjustedCloseChange'], 
       color='blue', alpha=0.5, width=40, label='Chevron Adjusted Close(negtive)')

#set a line of value 2 in ax1 as a baseline to see the deviation of inflation rate 
ax1.axhline(2, color='gray', linestyle='--')

#set the font size of tick labels
plt.tick_params(labelsize=10)  

#set title and fontsize
plt.title("Inflation vs. Chevron Stock Price", fontsize=15)

#get the handler and label of legend for ax1 and ax2
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()

#set the corresponding legend for both ax1 and ax2
#set the location to be upper left, which is location code 2
#set the pad between the axes and legend border to be 0
ax1.legend(handler1 + handler2, label1 + label2, loc=2, borderaxespad=0.)

#set x label to be Date
ax1.set_xlabel("Date")

#set y label for ax1
ax1.set_ylabel("Inflation Rate(%)")

#set y label for ax2
ax2.set_ylabel("Chevron Adjusted Price Change(%)")

#set ylim for ax1
ax1.set_ylim([-10, 10])

#set xlim for ax1, add a little time difference from both the left and right y-axis
ax1.set_xlim([min(df_change.index) - dt.timedelta(days=40),
              max(df_change.index) + dt.timedelta(days=100)])

#display the plot
plt.show()

### Conclusion:
- Both inflation rate and Chevron stock price are volatile. 
- The inflation rate is positive most of the time, but plummets after year 2008 to a negative value. It increases dramatically after year 2020 to an unprecedented high value.
- The Chevron stock price change has more positive values than negative values, meaning it is increasing generally.However, it plummets in 2020 and rebounces in 2022. 
- Although it appears that blue bars appear when inflation is bad, red bars are also present.

In [ ]:
# Let's draw a linear regression plot to examine the relationship between inflation rate and adjusted close change
sns.lmplot(x='InflationRate', y='AdjustedCloseChange', data=df_change)

#### Conclusion: 
The trend line shows that there is a positive correlation between inflation rate and chevson stock price change. When inflation rate increases, the adjusted close change increases. However, the correlation is very weak. When inflation rate is high, it doesn't necessarily mean the stock performance is good.

#### Draw a pairplot

In [ ]:
# create a column variable to store the columns that we want to retrieve
cols = ['InflationRate', 'GasPrice', 'Sales', 'NetIncome', 'Assets','Equity','Volume','AdjustedClose'] 

# create a pairplot to visualze the relationship between any two columns
sns.pairplot(df_merge[cols], height = 2.5)

# show the plot
plt.show()

#### Conclusion:
- The adjusted close has positive correlation with gas price, sales, net income, assets and equity. 
- Let's draw a heatmap to see how strong they are correlated.

In [ ]:
# plot a heatmap to explicitly show the correlation between any two features
# Compute a correlation matrix and convert to long-form
corr_mat = df_merge[cols].corr().stack().reset_index(name="correlation")
sns.set_style('whitegrid')
# Draw each cell as a scatter point with varying size and color
g = sns.relplot(data=corr_mat,x="level_0", y="level_1", hue="correlation", size="correlation",palette="vlag", hue_norm=(-1, 1), edgecolor=".7",height=10, sizes=(50, 250), size_norm=(-.2, .8),)

# Tweak the figure to finalize
g.set(xlabel="", ylabel="", aspect="equal")
g.despine(left=True, bottom=True)
g.ax.margins(.02)
for label in g.ax.get_xticklabels():
    label.set_rotation(90)
for artist in g.legend.legendHandles:
    artist.set_edgecolor(".7")
plt.title('Heatmap - Correlation', fontsize=30)

#### Conlusion:
- The Chevron sales has a strong correlation with gas price.
- The adjusted close has moderate to strong correlation with gas price, assets, and equity.

## Machine Learning


### Build Linear Regression Model to Predict Stock Price

In [ ]:
# Build linear regression model using gas price, assets and equity as predictors
# Split data into predictors X and output y
predictors = ['GasPrice', 'Assets', 'Equity']
X = df_merge[predictors]
y = df_merge['AdjustedClose']

In [ ]:
# split the dataset into training and testing data to verify the model
# test_size indicates the % of data placed in the test split
#ramdom_state indicates the shuffling of the data before splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
# Initialize and fit model
lm = LinearRegression()
# train the linear regression model using the training data
lm.fit(X_train, y_train)

In [ ]:
# use lm.predict() to predict off the X_test set of the data
y_pred = lm.predict(X_test)
y_pred

In [ ]:
#create a scatterplot of the real test values versus the predicted values
plt.figure(figsize=(15, 8))
plt.scatter(x=y_test, y=y_pred)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
plt.grid()

#### Evaluate the model

In [ ]:
#calculate the Mean Absolute Error, Mean Squared Error, and the Root Mean Squared Error
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

### Load Data / Features

In [ ]:
# load final df
dfmerge3 = pd.read_csv('data/final_df.csv',index_col=0)

# conversions
dfmerge4 = dfmerge3
dfmerge4.year = dfmerge3.year.astype(float)
dfmerge4.month = dfmerge3.month.astype(float)
dfmerge4.volume = dfmerge3.volume/1000000
dffinal = dfmerge4
dffinal.info()

In [ ]:
############################## MANUAL Train Test Split ##############################
# Since this is Time Series, we need split manually
# calculate train_size and test_size
train_size = int(len(dffinal)*0.8)
test_size = len(dffinal)-train_size

# split df 
traindf, testdf = dffinal.iloc[0:train_size], dffinal.iloc[train_size:]

# columns needed
featureColumns = ['volume','netIncome','equity','inflation','gas']
targetColumns = ['adjustedClose']

# Since data has wide range, apply Standard Scale
# X Scaler
featureScaler = StandardScaler()
X_train = traindf[featureColumns] = featureScaler.fit_transform(traindf[featureColumns].to_numpy())
X_test = testdf[featureColumns] = featureScaler.transform(testdf[featureColumns].to_numpy())

# y Scaler
targetScaler = StandardScaler()
y_train = traindf[targetColumns] = targetScaler.fit_transform(traindf[targetColumns])
y_test = testdf[targetColumns] = targetScaler.transform(testdf[targetColumns])



In [ ]:
# Convert dimension for LSTM
Xt, yt = convSeq(traindf[featureColumns], traindf[targetColumns],6)

Xv, yv = convSeq(testdf[featureColumns], testdf[targetColumns] ,6)

print(X_train.shape)
print(Xt.shape)

### Machine Learning

In [ ]:
linear = LinearRegression()
ridge = Ridge()
randomForest = RandomForestRegressor()
xgb = XGBRegressor()


In [ ]:
runML(linear)
runML(ridge)
runML(randomForest)
runML(xgb)

In [ ]:
# Since XGB performs best, use xgb calculate y_predict
ypredict = xgb.predict(XtestScale)

a = []
for i in ytestScale:
    a.append(float(i))

# graph it
plt.figure(figsize=(12,6))
sns.scatterplot(x=np.array(a),y=ypredict)
plt.title('Actual vs Predicted Price')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')


### Deep Learning

#### ANN

In [ ]:
# since ANN takes two dimension, we use X_train
# Features and optimizers
optimizer = 'rmsprop'

# Model
ann = Sequential()

# Hidden layer 1
ann.add(Flatten(input_shape= (traindf[featureColumns].shape[1], 1)))
ann.add(Dense(16))
ann.add(LeakyReLU(alpha=0.05))

# Hidden layer 2
ann.add(Dense(4))
ann.add(LeakyReLU(alpha=0.05))

# output layer
ann.add(Dense(1))

### compile the model using: optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']
ann.compile(loss='mse', optimizer=optimizer, metrics=['mse','mae'])

# model summary
ann.summary()

In [ ]:
history = ann.fit(traindf[featureColumns], traindf[targetColumns], epochs=50, batch_size=2, validation_data=(testdf[featureColumns], testdf[targetColumns]),verbose=1)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(ann.history.history['loss'][:])
plt.plot(ann.history.history['mae'][:])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss (mse)')
plt.legend(['Train', 'Test'], loc='upper right')

In [ ]:
# NN - this model takes in 3 dimension
inputs = Input(shape=(Xt.shape[1], Xt.shape[2]))
x = Flatten()(inputs)
x = Dense(16, activation=LeakyReLU(alpha=0.05))(x)
outputs = Dense(1)(x)
model = Model(inputs, outputs)

model.summary()

model.compile(optimizer=optimizer, loss="mse", metrics=['mse',"mae"])
history = model.fit(Xt, yt, epochs=50)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(model.history.history['loss'][:])
plt.plot(model.history.history['mae'][:])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss (mse)')
plt.legend(['Train', 'Test'], loc='upper right')

#### LSTM

In [ ]:
optimizer = 'adam'

# Model
lstm = Sequential()

# # Hideen layers
lstm.add(LSTM(units=16,activation = LeakyReLU(alpha=0.05), return_sequences = True, input_shape= (Xt.shape[1], Xt.shape[2])))
lstm.add(LSTM(units=8,activation = LeakyReLU(alpha=0.05),input_shape= (Xt.shape[1], 16)))
lstm.add(Dense(units=3, activation = LeakyReLU(alpha=0.05)))
lstm.add(Dense(units=1, activation = LeakyReLU(alpha=0.05)))


######## Does not work if i try to save model) ########
# checkpoint = ModelCheckpoint('lstm.h5', monitor='mse', save_best_only=True, verbose=1)
# lstm.compile(optimizer=optimizer, loss='mse',metrics=['mse','mae'],callbacks=[checkpoint])




lstm.compile(optimizer=optimizer, loss='mse',metrics=['mse','mae'])
lstm.summary()
history = lstm.fit(Xt, yt, epochs=50, batch_size=2,verbose=1)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(lstm.history.history['loss'][:])
plt.plot(lstm.history.history['mae'][:])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss (mse)')
plt.legend(['Train', 'Test'], loc='upper right')

In [ ]:
inputs = Input(shape=(Xt.shape[1], Xt.shape[2]))
x = LSTM(16)(inputs)
outputs = Dense(1)(x)
model = Model(inputs, outputs)

model.compile(optimizer="rmsprop", loss="mse", metrics=['mse',"mae"])
history = model.fit(Xt,yt,epochs=50)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(model.history.history['loss'][:])
plt.plot(model.history.history['mae'][:])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss (mse)')
plt.legend(['Train', 'Test'], loc='upper right')

#### RNN Bidirectional LSTM

In [ ]:
biLSTM = Sequential()

biLSTM.add(Bidirectional(LSTM(units=64,input_shape=(Xt[1],Xt[2]))))
biLSTM.add(Dense(units=32, activation='selu'))
biLSTM.add(Dense(units=16, activation='selu'))
biLSTM.add(Dense(units=8, activation='selu'))
biLSTM.add(Dense(units=1, activation='selu'))

biLSTM.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])

# biLSTM.summary()

history = biLSTM.fit(Xt, yt, epochs=50, batch_size=2,verbose=1)

In [ ]:
biLSTM.summary()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(biLSTM.history.history['loss'][:])
plt.plot(biLSTM.history.history['mae'][:])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss (mse)')
plt.legend(['Train', 'Test'], loc='upper right')

#### Hyperparameters

In [ ]:
def create_keras_model_to_find_optimizer(optimizer='adam'):
    # Model
    keras_model = Sequential()

    # Hidden layer 1
    keras_model.add(Flatten(input_shape= (traindf[featureColumns].shape[1], 1)))
    keras_model.add(Dense(16))
    keras_model.add(LeakyReLU(alpha=0.05))

    # Hidden layer 2
    keras_model.add(Dense(4))
    keras_model.add(LeakyReLU(alpha=0.05))

    # output layer
    keras_model.add(Dense(1))
    # Compile model
    keras_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])
    return keras_model
    
keras_model_02 = KerasClassifier(model=create_keras_model_to_find_optimizer, epochs=30, batch_size=2, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'GD', 'Adam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=keras_model_02, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(Xt, yt)

In [ ]:
ls_model = KerasClassifier(build_fn=hyperLSTM,verbose=1)
# units = [128, 64, 32, 16, 8]
# activation = ['selu', 'elu',LeakyReLU(alpha=0.05),'tanh','swish']
optimizer = ['SGD', 'GD', 'Adam', 'rmsprop']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=ls_model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(Xt, yt)

#### Predict

In [ ]:
# y_predict using Bidirectional LSTM model
y_predict = biLSTM.predict(Xv)
yt_inverse = targetScaler.inverse_transform(yt.reshape(1,-1))
yv_inverse = targetScaler.inverse_transform(yv.reshape(1,-1))
y_predict_inv = targetScaler.inverse_transform(y_predict)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(yv_inverse.flatten(),marker = '*', label='test')
plt.plot(y_predict_inv.flatten(),marker = '*', label='predict')
plt.legend()

## Conclusion


## References


----------
CIS43 Project Link<br>
https://colab.research.google.com/drive/1OS0i9lEitkL7PTE7TpLuWVoqkjS0BPYC
<br>

Load local file onto colab<br>
https://www.youtube.com/watch?v=6HFlwqK3oeo
<br>
<br>

----------
Inflation Data<br>
https://www.usinflationcalculator.com/inflation/current-inflation-rates/#:~:text=The%20annual%20inflation%20rate%20for,at%208%3A30%20a.m.%20ET.
<br>
<br>
Gas Prices<br>
https://www.kaggle.com/datasets/mruanova/us-gasoline-and-diesel-retail-prices-19952021
<br>
<br>